In [13]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------  11.0/11.1 MB 59.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 53.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   -------------- ------------------------- 14.7/41.0 MB 73.9 MB/s eta 0:00:01
   ----------------------------- ---------- 29.9/41.0 MB 73.7 MB/s eta 0:00:01
   ---------------------------------------  40.9/41.0 MB 73.7 MB/s eta 0:00:01
   ---------------------------------------- 41.0/41.0 MB 59.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pandas transformers

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   - -------------------------------------- 7.9/204.1 MB 40.3 MB/s eta 0:00:05
   ---- ----------------------------------- 22.5/204.1 MB 57.4 MB/s eta 0:00:04
   ------- -------------------------------- 38.5/204.1 MB 63.3 MB/s eta 0:00:03
   ---------- ----------------------------- 53.5/204.1 MB 65.9 MB/s eta 0:00:03
   ------------ --------------------------- 62.9/204.1 MB 62.6 MB/s eta 0:00:03
   --------------- ------------------------ 79.2/204.1 MB 63.7 MB/s eta 0:00:02
   ------------------ --------------------- 94.1/204.1 MB 65.2 MB/s eta 0:00:02
   --------------------- ----------------- 110.1/204.1 MB 66.3 MB/s eta 0:00:02
   ----------------------- --------------- 125.0/204.1 MB 67.1 MB/s eta 0:00:02
   -------------------------- ------------ 140.8/204.1 MB 67.7 MB/s eta 0:00:01
   ----------------------------- --------- 156.5/204.1 MB 68.3 MB/s eta 0:00:01
   -------------------------------- ------ 171.4/2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 
#Run if you have gpu

Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 6.1/6.1 MB 45.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB 71.7 MB/s eta 0:00:35
   ---------------------------------------- 0.0/2.5 GB 72.5 MB/s eta 0:00:35
    --------------------------------------- 0.0/2.5 GB 72.7 MB/s eta 0:00:34
    --------------------------------------- 0.1/2.5 GB 72.8 MB/s eta 0:00:34
   - -------------------------------------- 0.1/2.5 GB 72.9 MB/s eta 0:00:34
   - -------------------------------------- 0.1/2.5 GB 72.9 MB/s eta 0:00:33
   - -------------------------------------- 0.1/2.5 GB 72.9 MB/s eta 0:00:33
   - -------------------------------------- 0.1/2.5 GB 72.9 MB/s eta 0:00:33
   -- ------------------------------------- 0.1/2.5 GB 72.9 MB/s eta 0:00:33
   -- ---------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from collections import Counter
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForMaskedLM, BertConfig, AdamW, get_linear_schedule_with_warmup
from torch.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import os
import json
import math


save_path = "saved_models/viral_bert"
os.makedirs(save_path, exist_ok=True)  

def save_model(model, epoch):
    """Save model checkpoint."""
    torch.save(model.state_dict(), f"{save_path}/bert_epoch_{epoch}.pt")
    print(f"Model saved at epoch {epoch}")

def load_model(model, epoch):
    """Load model checkpoint."""
    model.load_state_dict(torch.load(f"{save_path}/bert_epoch_{epoch}.pt"))
    model.to(device)
    print(f"Loaded model from epoch {epoch}")


In [ ]:

# Load CSV file
def load_genomes_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    sequences = df["Genomic Sequence"].dropna().tolist()  
    return sequences

# Example Usage
csv_file = "../Datasets/coronaviridae_sequences_with_genomes_final.csv"
genome_sequences = load_genomes_from_csv(csv_file)
print(f"Loaded {len(genome_sequences)} genome sequences.")

def kmer_tokenizer(sequence, k=6):
    """Generate k-mers from a sequence."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]


Loaded 5606 genome sequences.


In [5]:

# Build Vocabulary
def build_kmer_vocab(sequences, k=6, vocab_size=30000):
    kmer_counts = Counter()
    for seq in sequences:
        kmer_counts.update(kmer_tokenizer(seq, k))
    most_common_kmers = [kmer for kmer, _ in kmer_counts.most_common(vocab_size - 2)]
    
    vocab = {"[PAD]": 0, "[MASK]": 1}
    vocab.update({kmer: i+2 for i, kmer in enumerate(most_common_kmers)})
    return vocab

vocab = build_kmer_vocab(genome_sequences, k=6, vocab_size=30000)
print(f"Vocabulary size: {len(vocab)}")



Vocabulary size: 26609


In [6]:

# Encode Sequences
def encode_sequence(sequence, vocab, k=6):
    kmers = kmer_tokenizer(sequence, k)
    return [vocab.get(kmer, 0) for kmer in kmers]

encoded_sequences = [encode_sequence(seq, vocab) for seq in genome_sequences]

# Create Overlapping Windows
def create_overlapping_windows(sequence, window_size=512, stride=256):
    windows = [sequence[i:i+window_size] for i in range(0, len(sequence), stride)]
    if len(windows[-1]) < window_size:
        windows[-1] += [0] * (window_size - len(windows[-1]))  # Pad last window
    return windows

windowed_sequences = [create_overlapping_windows(seq) for seq in encoded_sequences]
flattened_windows = [win for seq in windowed_sequences for win in seq]
print(f"Total windows: {len(flattened_windows)}")

Total windows: 634036


In [7]:

# Train/Validation/Test Split
train_data, test_data = train_test_split(flattened_windows, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}, Test size: {len(test_data)}")


Train size: 513568, Validation size: 57064, Test size: 63404


In [8]:

# Dataset Class
class GenomicDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        input_ids = torch.tensor(self.sequences[idx])
        return input_ids

def collate_fn(batch):
    input_ids = [torch.tensor(seq) for seq in batch]
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=vocab["[PAD]"])
    attention_mask = (input_ids != vocab["[PAD]"]).long()
    return input_ids, attention_mask

train_dataset = GenomicDataset(train_data)
val_dataset = GenomicDataset(val_data)
test_dataset = GenomicDataset(test_data)

In [9]:

batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [ ]:
import torch
print(torch.cuda.is_available()) 
print(torch.cuda.device_count())  # Should print the number of GPUs, e.g., 1
print(torch.cuda.get_device_name(0))  # Should print the name of your GPU (e.g., RTX 3070)


True
1
NVIDIA GeForce RTX 3070


In [12]:
import torch.nn as nn
# Model Configuration
config = BertConfig(vocab_size=len(vocab), hidden_size=288, num_hidden_layers=4, num_attention_heads=6, max_position_embeddings=512)
model = BertForMaskedLM(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Regularization and Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
total_steps = len(train_dataloader) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
criterion = nn.CrossEntropyLoss()

# Training Loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}")
    
    for input_ids, attention_mask in progress_bar:
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} | Avg Training Loss: {avg_loss:.4f}")
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for input_ids, attention_mask in val_dataloader:
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            val_loss += outputs.loss.item()
    
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1} | Validation Loss: {avg_val_loss:.4f}")

# Evaluate on Test Set
model.eval()
test_loss = 0
with torch.no_grad():
    for input_ids, attention_mask in test_dataloader:
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        test_loss += outputs.loss.item()

avg_test_loss = test_loss / len(test_dataloader)
print(f"Test Loss: {avg_test_loss:.4f}")

# Save Model
model.save_pretrained("viral_bert_final_gen_2")


Epoch 1/5:   0%|          | 0/64196 [00:00<?, ?it/s]C:\Users\haris\AppData\Local\Temp\ipykernel_20772\3346798437.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = [torch.tensor(seq) for seq in batch]
Epoch 1/5: 100%|██████████| 64196/64196 [1:24:26<00:00, 12.67it/s, loss=2.01e-7] 


Epoch 1 | Avg Training Loss: 0.0465
Epoch 1 | Validation Loss: 0.0003


Epoch 2/5: 100%|██████████| 64196/64196 [1:24:05<00:00, 12.72it/s, loss=1.16e-9] 


Epoch 2 | Avg Training Loss: 0.0003
Epoch 2 | Validation Loss: 0.0001


Epoch 3/5: 100%|██████████| 64196/64196 [1:23:38<00:00, 12.79it/s, loss=5.82e-11]


Epoch 3 | Avg Training Loss: 0.0001
Epoch 3 | Validation Loss: 0.0001


Epoch 4/5: 100%|██████████| 64196/64196 [1:23:38<00:00, 12.79it/s, loss=0]       


Epoch 4 | Avg Training Loss: 0.0000
Epoch 4 | Validation Loss: 0.0000


Epoch 5/5: 100%|██████████| 64196/64196 [1:24:24<00:00, 12.68it/s, loss=6.69e-10]


Epoch 5 | Avg Training Loss: 0.0000
Epoch 5 | Validation Loss: 0.0000
Test Loss: 0.0000


In [9]:
# Model Configuration
config = BertConfig(
    vocab_size=len(vocab),
    hidden_size=288,
    num_hidden_layers=4,
    num_attention_heads=6,  # 288 is divisible by 6
    max_position_embeddings=512
)
model = BertForMaskedLM(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
total_steps = len(dataloader) * 5  # 5 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
# FP16 Mixed Precision Training
scaler = GradScaler()

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

    for input_ids, attention_mask, labels in progress_bar:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with autocast(device_type='cuda'):
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} | Avg Loss: {avg_loss:.4f}")
    
    # Save Model
    model.save_pretrained(f"viral_bert_epoch_{epoch+1}")


c:\Users\haris\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/5:   0%|          | 0/79255 [00:00<?, ?it/s]C:\Users\haris\AppData\Local\Temp\ipykernel_63104\3552041536.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = [torch.tensor(seq) for seq in input_ids]
C:\Users\haris\AppData\Local\Temp\ipykernel_63104\3552041536.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = [torch.tenso

Epoch 1 | Avg Loss: 0.6779


Epoch 2/5: 100%|██████████| 79255/79255 [1:05:39<00:00, 20.12it/s, loss=0.0192] 


Epoch 2 | Avg Loss: 0.0374


Epoch 3/5: 100%|██████████| 79255/79255 [1:05:47<00:00, 20.08it/s, loss=0.0251] 


Epoch 3 | Avg Loss: 0.0223


Epoch 4/5: 100%|██████████| 79255/79255 [1:05:50<00:00, 20.06it/s, loss=0.0148] 


Epoch 4 | Avg Loss: 0.0172


Epoch 5/5: 100%|██████████| 79255/79255 [1:05:53<00:00, 20.05it/s, loss=0.00798]


Epoch 5 | Avg Loss: 0.0147


In [18]:
from sklearn.model_selection import train_test_split

# Assuming `encoded_sequences` contains your preprocessed sequences
train_seqs, val_seqs = train_test_split(encoded_sequences, test_size=0.2, random_state=42)

# Pad sequences to max length
max_length = max(len(seq) for seq in encoded_sequences)
val_seqs_padded = [seq + [0] * (max_length - len(seq)) for seq in val_seqs]

# Convert to tensors
val_input_ids = torch.tensor(val_seqs_padded)

# Create DataLoader for validation
from torch.utils.data import TensorDataset, DataLoader
batch_size = 8
val_dataset = TensorDataset(val_input_ids)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"Validation set size: {len(val_input_ids)}")


Validation set size: 1122


In [21]:
# Load CSV file
def load_genomes_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    sequences = df["Genomic Sequence"].dropna().tolist()  # Ensure no NaN values
    return sequences

# Example Usage
csv_file = "../Datasets/coronaviridae_test_run.csv"
genome_sequences = load_genomes_from_csv(csv_file)
print(f"Loaded {len(genome_sequences)} genome sequences.")

def kmer_tokenizer(sequence, k=6):
    """Generate k-mers from a sequence."""
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

vocab = build_kmer_vocab(genome_sequences, k=6, vocab_size=30000)
print(f"Vocabulary size: {len(vocab)}")


encoded_sequences = [encode_sequence(seq, vocab) for seq in genome_sequences]

Loaded 10 genome sequences.
Vocabulary size: 3754


In [22]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch

# Assuming `encoded_sequences` contains your preprocessed sequences
train_seqs, val_seqs = train_test_split(encoded_sequences, test_size=0.2, random_state=42)

# Pad sequences to max length
max_length = 512  # Make sure this matches your model's max_position_embeddings
train_seqs_padded = [seq + [0] * (max_length - len(seq)) if len(seq) < max_length else seq[:max_length] for seq in train_seqs]
val_seqs_padded = [seq + [0] * (max_length - len(seq)) if len(seq) < max_length else seq[:max_length] for seq in val_seqs]

# Convert to tensors
train_input_ids = torch.tensor(train_seqs_padded)
val_input_ids = torch.tensor(val_seqs_padded)

# Labels for MLM are the same as input_ids, but we will mask out padding tokens
train_labels = train_input_ids.clone()
val_labels = val_input_ids.clone()

# Create DataLoader for validation
batch_size = 8
val_dataset = TensorDataset(val_input_ids, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Evaluate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

val_losses = []
with torch.no_grad():
    for batch in tqdm(val_dataloader, desc="Evaluating"):
        input_ids, labels = batch
        input_ids, labels = input_ids.to(device), labels.to(device)

        # Forward pass
        with autocast(device_type='cuda'):
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
        
        val_losses.append(loss.item())

# Calculate average validation loss
avg_val_loss = sum(val_losses) / len(val_losses)
print(f"Validation Loss: {avg_val_loss:.4f}")


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.91it/s]

Validation Loss: 0.0416
